In [12]:
import json

In [13]:
with open("intents.json") as file:
	data = json.load(file)

In [14]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leinad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
import numpy
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow import keras

In [16]:
# prompt = input('enter your prompt: ')
# # issue = ['engine issue', 'battery issue', 'break problem', 'overheating']
# for intent in data["intents"]:
#     for pattern in intent["patterns"]:
#         if prompt in pattern:
#             print(intent["tag"])

In [17]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [18]:
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

In [19]:
try:
	model.load("model.tflearn")
except:
	model = tflearn.DNN(net)
	model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
	model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from c:\Users\Leinad\Documents\Tech_Villain\Leinadpython\CarFault\model.tflearn


In [20]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [21]:
def chat():
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break
		
		for intent in data["intents"]:
			for pattern in intent["patterns"]:
				if inp in pattern:
					print(intent["tag"])
		
		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']	
			print(random.choice(responses))
			print("\n")
		else:
			print("I didnt get that, try again")

In [25]:
chat()

Check the coolant level and add more coolant if needed
Check the engine oil and add more oil if needed
Check the radiator:  The radiator consists of a fan and some pipes, The radiator fan fails and sometimes it just runs out of water. You may refill it, and if the fan is not working, then call your mechanic right away.


OVERHEATING ENGINE
Check the coolant level and add more coolant if needed
Check the engine oil and add more oil if needed
Check the radiator:  The radiator consists of a fan and some pipes, The radiator fan fails and sometimes it just runs out of water. You may refill it, and if the fan is not working, then call your mechanic right away.




In [ ]:
for intent in data["intents"]:
			for pattern in intent["patterns"]:
				if inp in pattern:
					print(f'This is most likely the fault with your car {intent["tag"]}')